In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2
import msmrd2.tools.quaternions as quats
import msmrd2.visualization as msmrdvis
from msmrd2.potentials import patchyParticleAngular2
from msmrd2.integrators import overdampedLangevin as odLangevin

In [ ]:
# Define particle list with between two and five particles to estimate the diffusion
# of dimers/ trimers / cuatrimers? and pentamer
np.random.seed(seed=1) # seed 1 good for pentamer 
numparticles = 5
boxsize = 3
D = 1.0
Drot = 1.0
pyPartlist = [] 

# Position List for pentamer IC
th = 2*np.pi/5.0
thextra = np.pi/2 + th/2.0
thextra = np.pi
for i in range(numparticles):
    position = 0.85 * np.array([np.cos(th*i),np.sin(th*i),0.0])
    orientation = np.array([np.cos(0.5*(thextra - th*i)),0,0,np.sin(0.5*(thextra - th*i))])
    part = msmrd2.particle(D, Drot, position, quats.conjugate(orientation))
    pyPartlist.append(part)

In [ ]:
# Create list of particles that can be read from msmrd
# Note the particles in this list will be independent from the python list.
partlist = msmrd2.integrators.particleList(pyPartlist)

In [ ]:
# Over-damped Langevin integrator definition
dt = 0.00001 #0.000005
seed = -1 #1 #-1 # Negative seed, uses random device as seed
bodytype = 'rigidbody'
integrator = odLangevin(dt, seed, bodytype) 

In [ ]:
# Define boundary (choose either spherical or box)
# sphericalBoundary = msmrd2.sphere(radius,'reflective')
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'reflective')
integrator.setBoundary(boxBoundary)

In [ ]:
# Define Patchy Particle potential
sigma = 1.0
strength = 160 #100# 60 #200.0
angularStrength = 20 #10 #200.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]
potentialPatchyParticleAngular2 = patchyParticleAngular2(sigma, strength, angularStrength, patchesCoordinates)
integrator.setPairPotential(potentialPatchyParticleAngular2)

In [ ]:
#Integrate particle list and print only positions 
timesteps = 100000 #20000000
stride = 250 #250 #1000
datafile  = open('../../data/vmd/pentamerTest_' + str(numparticles) + 'particles.xyz', 'w')
for i in range(timesteps):
    if i%stride == 0:
        datafile.write(str(3*len(partlist)) + '\n')
        datafile.write(str(0) + '\n')
    for j, part in enumerate(partlist):
        if i%stride == 0:
            v0 = part.position
            v1 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[0], part.orientation)
            v2 = v0 + 0.5*sigma*quats.rotateVec(patchesCoordinates[1], part.orientation)
            datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
            datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
    integrator.integrate(partlist)
    if i%10000 == 0:
        print("Percentage complete: ", 100*i/timesteps, "%", end="\r")
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticles(numparticles = numparticles, 
                                    outfname = 'pentamerTest_' + str(numparticles) + 'particles', 
                                    tclfname = '../../data/vmd/pentamerTest_' + str(numparticles) + 'particles_2vmd.tcl')
print("Percentage complete: ", 100, " %")